 ### **Question D | Bouchaud Transitory Impact Model**

> With the dataset and the framework provided for TD4, estimate all the parameters of Bouchaud's price
impact model. Comment the obtained values. Is this model well specified?

In [2]:
# Open TD4 data
import pandas as pd
df = pd.read_excel("Dataset TD4.xlsx")
df

,transaction date (1=1day=24 hours),bid-ask spread,volume of the transaction (if known),Sign of the transaction,Price (before transaction)
0,0.000202,0.1100,8.0,-1,100.000
1,0.001070,0.1030,NaN,1,99.984
2,0.001496,0.1015,NaN,-1,100.029
3,0.003336,0.0920,NaN,1,99.979
4,0.003952,0.1106,NaN,1,100.060
...,...,...,...,...,...
996,0.981441,0.0834,79.0,1,101.070
997,0.981875,0.1010,NaN,-1,101.120
998,0.986784,0.1007,NaN,-1,100.998
999,0.991232,0.1153,3.0,-1,100.958


In [ ]:
# Rename the columns for simplicity
df.rename(columns={'transaction date (1=1day=24 hours)': 'date', 'bid-ask spread': 'spread', 'volume of the transaction (if known)': 'volume', 'Sign of the transaction': 'sign', 'Price (before transaction)': 'price'}, inplace=True)
df

,date,spread,volume,sign,price,date
0,0.000202,0.1100,8.0,-1,100.000,0.000202
1,0.001070,0.1030,NaN,1,99.984,0.001070
2,0.001496,0.1015,NaN,-1,100.029,0.001496
3,0.003336,0.0920,NaN,1,99.979,0.003336
4,0.003952,0.1106,NaN,1,100.060,0.003952
...,...,...,...,...,...,...
996,0.981441,0.0834,79.0,1,101.070,0.981441
997,0.981875,0.1010,NaN,-1,101.120,0.981875
998,0.986784,0.1007,NaN,-1,100.998,0.986784
999,0.991232,0.1153,3.0,-1,100.958,0.991232


Bouchaud's transitory impact model is specified as:
$$p_t = p_{-\infty} + \sum^{t-1}_{s=-\infty}G(t-s)\epsilon_s S_s V^r_s$$

Where:
- $S$ is the bid-ask spread;
- $\epsilon$ is -1 or 1 depending on whether the transaction is a buy (price = ask) or a sell (price = bid) on the market;
- $V$ is the volume of the transaction, with $r$ close to zero to have a concave function of the volume;
- $G$ is a function worth 0 on $\mathbb{R}^-$, which can be interpreted as the impact of a single order.
